# Случайный лес

Лео Брейман нашел применение бутстрэпу не только в статистике, но и в машинном обучении. Он вместе с Адель Катлер усовершенстовал алгоритм случайного леса, который был предложенный [Хо](http://ect.bell-labs.com/who/tkh/publications/papers/odt.pdf), добавив к первоначальному варианту построение некоррелируемых деревьев на основе CART, в сочетании с методом случайных подпространств и бэггинга. 

Решающие деревья являются хорошим семейством базовых классификаторов для бэггинга, поскольку они достаточно сложны и могут достигать нулевой ошибки на любой выборке. Метод случайных подпространств позволяет снизить коррелированность между деревьями и избежать переобучения. Базовые алгоритмы обучаются на различных подмножествах признакового пространства, которые также выбираются случайным образом.
Ансамбль моделей, использующих метод случайного подпространства, можно построить, используя следующий алгоритм:

1. Пусть количество объектов на тренировочной выборке равно $\large N$, а количество признаков $\large D$.
2. Выбираем $\large L$ - число отдельных моделей в ансамбле.
3. Для каждой отдельной модели $\large l$ берем $\large dl (dl < D) $ как число признаков для $\large l$ . Обычно для всех моделей используется только одно значение $\large dl$.
4. Для каждой отдельной модели $\large l$ создаем обучающую выборку, выбрав $\large dl$-признаков из $\large D$ с замещением и обучаем модель.

Теперь, чтобы применить модель ансамбля к новому объекту, объединяем результаты отдельных $\large L$ моделей мажоритарным голосованием или путем комбинирования апостериорных вероятностей.


## Алгоритм

Алгоритм построения случайного леса, состоящего из $\large N$ деревьев, выглядит следующим образом:
* Для каждого $\large n = 1, \dots, N$:
     * Сгенерировать выборку $\large X_n$ с помощью bootstrap.
     * Построить решающее дерево $\large b_n$ по выборке $\large X_n$:
         — по заданному критерию мы выбираем лучший признак, делаем разбиение в дереве по нему и так до исчерпания выборки
         — дерево строится, пока в каждом листе не более $\large n_\text{min}$ объектов или пока не достигнем определенной глубины дерева
         — при каждом разбиении сначала выбирается $\large m$ случайных признаков из $\large n$ исходных, 
         и оптимальное разделение выборки ищется только среди них.
         
Итоговый классификатор $\large a(x) = \frac{1}{N}\sum_{i = 1}^N b_i(x)$, простыми словами — для задачи кассификации мы выбираем решение голосованием по большинству, а в задаче регрессии — средним.

Рекомендуется в задачах классификации брать $\large m = \sqrt{n}$, а в задачах регрессии — $\large m = \frac{n}{3}$, где $\large n$ — число признаков. Также рекомендуется в задачах классификации строить каждое дерево до тех пор, пока в каждом листе не окажется по одному объекту, а в задачах регрессии — пока в каждом листе не окажется по пять объектов.

Таким образом, случайный лес — это бэггинг над решающими деревьями, при обучении которых для каждого разбиения признаки выбираются из некоторого случайного подмножества признаков.

## Параметры


Метод случайного леса реализован в библиотеке машинного обучения [scikit-learn](http://scikit-learn.org/stable/) двумя классами `RandomForestClassifier` и `RandomForestRegressor`.

Полный список параметров случайного леса для задачи регрессии:

**n_estimators** — число деревьев в "лесу" (по дефолту – 10)

**criterion** — функция, которая измеряет качество разбиения ветки дерева (по дефолту — "mse" , так же можно выбрать "mae")

**max_features** — число признаков, по которым ищется разбиение. Вы можете указать конкретное число или процент признаков, либо выбрать из доступных значений: "auto" (все признаки), "sqrt", "log2". По дефолту стоит "auto".

**max_depth** — максимальная глубина дерева  (по дефолту глубина не ограничена)

**min_samples_split** — минимальное количество объектов, необходимое для разделения внутреннего узла. Можно задать числом или процентом от общего числа объектов (по дефолту — 2)

**min_samples_leaf** — минимальное число объектов в листе. Можно задать числом или процентом от общего числа объектов (по дефолту — 1)

**min_weight_fraction_leaf** — минимальная взвешенная доля от общей суммы весов (всех входных объектов) должна быть в листе (по дефолту имеют одинаковый вес)

**max_leaf_nodes** — максимальное количество листьев (по дефолту нет ограничения)

**min_impurity_split** — порог для остановки наращивания дерева (по дефолту 1е-7)

**bootstrap** — применять ли бустрэп для построения дерева (по дефолту True)

**oob_score** — использовать ли out-of-bag объекты для оценки R^2 (по дефолту False)

**n_jobs** — количество ядер для построения модели и предсказаний (по дефолту 1, если поставить -1, то будут использоваться все ядра)

**random_state** — начальное значение для генерации случайных чисел (по дефолту его нет, если хотите воспроизводимые результаты, то нужно указать любое число типа int

**verbose** — вывод логов по построению деревьев (по дефолту 0)

**warm_start** — использует уже натренированую модель и добавляет деревьев в ансамбль (по дефолту False)

Для задачи классификации все почти то же самое, мы приведем только те параметры, которыми `RandomForestClassifier` отличается от `RandomForestRegressor`

**criterion** — поскольку у нас теперь задача классификации, то по дефолту выбран критерий "gini" (можно выбрать "entropy")

**class_weight** — вес каждого класса (по дефолту все веса равны 1, но можно передать словарь с весами, либо явно указать "balanced", тогда веса классов будут равны их исходным частям в генеральной совокупности; также можно указать "balanced_subsample", тогда веса на каждой подвыборке будут меняться в зависимости от распределения классов на этой подвыборке.

Параметры, на которые в первую очередь стоит обратить внимание при построении модели:
- n_estimators — число деревьев в "лесу"
- criterion — критерий для разбиения выборки в вершине
- max_features — число признаков, по которым ищется разбиение
- min_samples_leaf — минимальное число объектов в листе
- max_depth — максимальная глубина дерева

## Важность признаков

Очень часто вы хотите понять свой алгоритм, почему он именно так, а не иначе дал определенный ответ. Или если не понять его полностью, то хотя бы какие переменные больше всего влияют на результат. Из случайного леса можно довольно просто получить данную информацию.

**Суть метода**

По данной картинке интуитивно понятно, что важность признака «Возраст»  в задаче кредитного скоринга выше, чем важность признака «Доход» . Формализуется это с помощью понятия прироста информации.
<img src="https://github.com/iakubovskii7/mlcourse.ai/blob/master/img/credit_scoring_toy_tree.gif?raw=true" align='center'>

Если построить много деревьев решений (случайный лес), то чем выше в среднем признак в дереве решений, тем он важнее в данной задаче классификации/регрессии. При каждом разбиении в каждом дереве улучшение критерия разделения (в нашем случае коэффициент Джини) — это показатель важности, связанный с переменной разделения, и накапливается он по всем деревьям леса отдельно для каждой переменной.

Давайте немного углубимся в детали. Среднее снижение точности, вызываемое переменной, определяется во время фазы вычисления out-of-bag ошибки. Чем больше уменьшается точность предсказаний из-за исключения (или перестановки) одной переменной, тем важнее эта переменная, и поэтому переменные с бо́льшим средним уменьшением точности более важны для классификации данных. Среднее уменьшение коэффициента Джини (или ошибки mse в задачах регрессии) является мерой того, как каждая переменная способствует однородности узлов и листьев в окончательной модели случайного леса. Каждый раз, когда отдельная переменная используется для разбиения узла, коэффициент Джини для дочерних узлов рассчитывается и сравнивается с коэффициентом исходного узла. Коэффициент Джини является мерой однородности от 0 (однородной) до 1 (гетерогенной). Изменения в значении критерия разделения суммируются для каждой переменной и нормируются в конце вычисления. Переменные, которые приводят к узлам с более высокой чистотой, имеют более высокое снижение коэффициента Джини.

А теперь представим все вышеописанное в виде формул. 
$$ \large VI^{T} = \frac{\sum_{i \in \mathfrak{B}^T}I \Big(y_i=\hat{y}_i^{T}\Big)}{\Big |\mathfrak{B}^T\Big |} - \frac{\sum_{i \in \mathfrak{B}^T}I \Big(y_i=\hat{y}_{i,\pi_j}^{T}\Big)}{\Big |\mathfrak{B}^T\Big |} $$

$ \large \hat{y}_i^{(T)} = f^{T}(x_i)  $ — предсказание класса перед перестановкой/удалением признака
$ \large \hat{y}_{i,\pi_j}^{(T)} = f^{T}(x_{i,\pi_j})   $ — предсказание класса после перестановки/удаления признака
$ \large x_{i,\pi_j} = (x_{i,1}, \dots , x_{i,j-1}, \quad x_{\pi_j(i),j}, \quad x_{i,j+1}, \dots , x_{i,p})$
Заметим, что $ \large VI^{(T)}(x_j) = 0 $, если $ \large X_j $  не находится в дереве $ \large T $ 

Расчет важности признаков в ансамбле:
— ненормированные 
$$ \large VI(x_j) = \frac{\sum_{T=1}^{N}VI^{T}(x_j)}{N} $$

— нормированные 
$$ \large z_j = \frac{VI(x_j)}{\frac{\hat{\sigma}}{\sqrt{N}}} $$

## Преимущества и недостатки случайного леса

**Плюсы**:
 - имеет высокую точность предсказания, на большинстве задач будет лучше линейных алгоритмов; точность сравнима с точностью бустинга
 - практически не чувствителен к выбросам в данных из-за случайного сэмлирования
 - нечувствителен к масштабированию (и вообще к любым монотонным преобразованиям) значений признаков, связано с выбором случайных подпространств
 - не требует тщательной настройки параметров, хорошо работает «из коробки». С помощью «тюнинга» параметров можно достичь прироста от 0.5 до 3% точности в зависимости от задачи и данных
 - способен эффективно обрабатывать данные с большим числом признаков и классов
 - одинаково хорошо обрабатывет как непрерывные, так и дискретные признаки
 - редко переобучается, на практике добавление деревьев почти всегда только улучшает композицию, но на валидации, после достижения определенного количества деревьев, кривая обучения выходит на асимптоту
 - для случайного леса существуют методы оценивания значимости отдельных признаков в модели
 - хорошо работает с пропущенными данными; сохраняет хорошую точность, если большая часть данных пропущенна
 - предполагает возможность сбалансировать вес каждого класса на всей выборке, либо на подвыборке каждого дерева
 - вычисляет близость между парами объектов, которые могут использоваться при кластеризации, обнаружении выбросов или (путем масштабирования) дают интересные представления данных
 - возможности, описанные выше, могут быть расширены до немаркированных данных, что приводит к возможноти делать кластеризацию и визуализацию данных, обнаруживать выбросы
 - высокая параллелизуемость и масштабируемость.
 
**Минусы**:
 - в отличие от одного дерева, результаты случайного леса сложнее интерпретировать
 - нет формальных выводов (p-values), доступных для оценки важности переменных
 - алгоритм работает хуже многих линейных методов, когда в выборке очень много разреженных признаков (тексты, Bag of words)
 - случайный лес не умеет экстраполировать, в отличие от той же линейной регрессии (но это можно считать и плюсом, так как не будет экстремальных значений в случае попадания выброса)
 - алгоритм склонен к переобучению на некоторых задачах, особенно на зашумленных данных
 - для данных, включающих категориальные переменные с различным количеством уровней, случайные леса предвзяты в пользу признаков с большим количеством уровней: когда у признака много уровней, дерево будет сильнее подстраиваться именно под эти признаки, так как на них можно получить более высокую точность
 - если данные содержат группы коррелированных признаков, имеющих схожую значимость для меток, то предпочтение отдается небольшим группам перед большими
 - больший размер получающихся моделей. Требуется $O(NK)$ памяти для хранения модели, где $K$ — число деревьев.

**Полезные источники:**
- 15 раздел книги “[Elements of Statistical Learning](https://web.stanford.edu/~hastie/Papers/ESLII.pdf)” Jerome H. Friedman, Robert Tibshirani, and Trevor Hastie
- [Блог](https://alexanderdyakonov.wordpress.com/2016/11/14/случайный-лес-random-forest/) Александра Дьяконова
- больше про практические применение случайного леса и других алгоритмов композиций в официальной документации [scikit-learn](http://scikit-learn.org/stable/modules/ensemble.html)
- [Курс](https://github.com/esokolov/ml-course-hse) Евгения Соколова по машинному обучению (материалы на GitHub). Есть дополнительные практические задания для углубления ваших знаний.

# Бустинг

Бустинг представляет собой жадный алгоритм построения композиции алгоритмов. Основная идея заключается в том, чтобы, имея множество относительно слабых алгоритмов обучения, построить их хорошую линейную комбинацию. Он похож на бэггинг тем, что базовый алгоритм обучения фиксирован. Отличие состоит в том, что обучение базовых алгоритмов для композиции происходит итеративно, и каждый следующий алгоритм стремится компенсировать недостатки композиции всех предыдущих алгоритмов.

На примере бустинга стало ясно, что хорошим качеством могут обладать сколь угодно сложные композиции классификаторов, при условии, что они правильно настраиваются. Это развеяло существовавшие долгое время представления о том, что для повышения обобщающей способности необходимо ограничивать сложность алгоритмов. 

Впоследствии этот феномен бустинга получил теоретическое обоснование. Оказалось, что взвешенное голосование не увеличивает эффективную сложность алгоритма, а лишь сглаживает ответы базовых алгоритмов. Эффективность бустинга объясняется тем, что по мере добавления базовых алгоритмов увеличиваются отступы обучающих объектов. Причём бустинг продолжает раздвигать классы даже после достижения безошибочной классификации обучающей выборки.

**Общая схема бустинга**:



## Градиентный бустинг
Метод градиентного бустинга в некотором смысле является обобщением остальных методов бустинга, поскольку он позволяет оптимизировать произвольную дифференцируемую функцию потерь. Данный алгоритм похож на метод градиентного спуска, применяемый для решения задач оптимизации. Основная идея заключается в том, что каждый следующий добавляемый в композицию алгоритм настраивается на остатки предыдущих алгоритмов.

Задача градиентного бустинга - обучение $N$-ой модели:

$$ \Large \frac{1} {\ell} \sum_{i=1}^{\ell} L (y_i, a_{N-1}(x_i) + b_N(x_i)) \rightarrow  \underset{b_N(x)}{min}$$

$\Large a_{N-1}(x_i)$ - прогноз алгоритма на $N-1$ шаге

Как посчитать и куда сдвигать $\large a_{N-1}(x_i)$, чтобы уменьшить ошибку? Взять производную!

$$ \Large s_{i}^{N} = -\frac{\partial}{\partial z} L(y_i, z) \ в \ точке \ \large z = a_{N-1}(x_i) $$

Знак этой производной будет показывать, в какую сторону сдвигать прогноз на $x_i$, чтобы уменьшить ошибку композиции на нем. Величина показывает, насколько сильно мы можем уменьшить ошибку при сдвиге предсказания. Если сдвиг $s_{i}^{N} > 0$, то прогноз увеличиваем, в противном случае - уменьшаем. 

Теперь наша задача немного меняется:

$$ \Large \frac{1} {\ell} \sum_{i=1}^{\ell} (b_N(x_i) - s_{i}^{N})^2 \rightarrow  \underset{b_N(x)}{min}$$

Пусть $\large L(y_i, z) = MSE$. Тогда $\large s_i^N = y_i - a_{N-1}(x_i)$. В итоге минимизируем следующую функцию потерь (`задача регрессии`):

$$ \Large \frac{1} {\ell} \sum_{i=1}^{\ell} [b_N(x_i) - (y_i - a_{N-1}(x_i))]^2 \rightarrow  \underset{b_N(x)}{min}$$

Пусть $\large L(y_i, z) = logloss$. Тогда минимизируем следующую функцию потерь (`задача классификации`):

$$ \Large \frac{1} {\ell} \sum_{i=1}^{\ell} [b_N(x_i) - \frac{y_i}{1 - exp(y_i a_{N-1}(x_i))} ]^2 \rightarrow  \underset{b_N(x)}{min}$$

Вспомним, что при бэггинге смещение не изменяется, а разброс снижается. В случае градиентного бустинга смещение базовых моделей снижается, но может возрасти дисперсия. В этом случае в качестве базовых моделей нужно брать `неглубокие` деревья. 

Еще одна проблема бустинга - из-за простоты базовых моделей. мы можем не найти то направление, куда нужно идти, чтобы снизить ошибку (например, базовая модель говорит нам об одном направлении движения, а в реальности нужно идти в другую сторону). В этом случае мы можем добавлять базовые модели с некоторым весом: $\large a_N(x) = a_{N-1}(x_i) + \eta b_N(x_i)$, $\eta \in (0;1]$ - длина шага. Здесь $\large \eta$ - регуляризация композиции. За счет этого вклад отдельных моделей в композицию снижается. Как правило, чем меньше $\eta$, тем больше деревьев. 

Еще один способ снизить переобучение моделей бустинга - обучать деревья на случайных подмножествах признаков (как и в случайном лесе). Помимо этого, можно еще случайно выкидывать часть наблюдений из выборки.

Ниже рассмотрим основные подходы, которые используются в градиентном бустинге: 

- ODT - oblivious decision trees

- Leaf-wise (best-first) tree growth

- Level-wise (depth-first) tree growth.

## ODT - oblivious decision trees

На уровне одного дерева мы используем один и тот же признак, потому что он ускоряет алгоритмы в несколько раз и в то же время не ухудшает результаты. Это происходит из-за того, что композиция `простых` моделей дает лучшие результаты. Количество одинаковых операций равно $n-1$, где $n$ - глубина дерева. Такие деревья сбалансированы, менее склонны к переобучению и позволяют значительно ускорить предсказание во время тестирования.

![](https://miro.medium.com/max/700/1*AjrRnwvBuu-zK8CvEfM29w.png)

Одним из самых популярных примеров является [CatBoost](https://catboost.ai/), созданный [Яндексом](https://en.wikipedia.org/wiki/Yandex). 

CatBoost - это высокопроизводительная библиотека с открытым исходным кодом для градиентного бустинга на деревьях решений.

CatBoost используется для поиска, рекомендательных систем, персональных помощников, самодвижущихся автомобилей, предсказания погоды и многих других задач в Яндексе и других компаниях, включая CERN, Cloudflare, такси Careem. Он имеет открытый исходный код и может быть использован любым желающим.

**Категориальные особенности** в CatBoost

CatBoost использует комбинации категориальных признаков в качестве дополнительных категориальных признаков, которые захватывают зависимости высокого порядка, такие как совместная информация ID пользователя и темы объявления в задаче предсказания рекламных кликов. Количество возможных комбинаций растет экспоненциально с ростом числа категориальных признаков в наборе данных, и обработать их все не представляется возможным. CatBoost строит комбинации жадным способом. А именно, для каждого разбиения дерева CatBoost объединяет (конкатенирует) все категориальные признаки (и их комбинации), уже использованные для предыдущих разбиений текущего дерева, со всеми категориальными признаками в наборе данных. Комбинации преобразуются в TS на лету.

CatBoost реализует алгоритм, который позволяет бороться с обычными предубеждениями градиентного бустинга. Существующие реализации сталкиваются со статистической проблемой - смещением предсказания. Эта проблема похожа на ту, что возникает при предварительной обработке категориальных переменных, которая была описана выше.

Команда Catboost разработала упорядоченный бустинг, модификацию стандартного алгоритма градиентного бустинга, которая позволяет избежать смещения (но это не точно).

Github этой библиотеки размещен [здесь](https://github.com/catboost).

[Установка](https://catboost.ai/docs/concepts/python-installation.html)

Основные **преимущества** CatBoost:

- отличное качество без подстройки параметров

- поддержка категориальных признаков (вам не нужна предварительная обработка категориальных признаков)

- быстрая и масштабируемая GPU-версия

- быстрое предсказание и повышенная точность

- вы можете написать собственную функцию потерь

- предоставляет инструменты для пакета Python, которые позволяют строить графики с различной статистикой обучения. Доступ к этой информации возможен как во время, так и после процедуры обучения

- его можно интегрировать в Tensorflow. Например, это распространенный случай объединения Catboost и Tensorflow. Нейронная сеть может быть использована для извлечения признаков для градиентного бустинга.

Основные **недостатки** CatBoost:

- несмотря на хорошее прогнозирование, как утверждает Яндекс, качество прогнозов во многих реальных задачах выглядит хуже, чем в случае других алгоритмов (см., например, [здесь](https://www.kdnuggets.com/2018/03/catboost-vs-light-gbm-vs-xgboost.html)).

- ...придумайте что-то еще


## Leaf-wise (best-first) tree growth

Данный метод подразумевает построение дерева до ***максимального количества листьев***. Среди всех вариантов листьев выбираем тот вариант, расщепление при котором сильно уменьшает ошибку.

![](https://miro.medium.com/max/700/1*P1uHwsMu_f0zGEvh4YK0kg.png)

Популярный пример такого вида бустинга - алгоритм [LightGBM](https://lightgbm.readthedocs.io/en/latest/Features.html). В 2017 году компания microsoft создала LightGBM как альтернативу использованию XGBoost. LightGBM может быть использован в Python, R и C++. 

Помимо `leaf-wise` подхода, основное его отличие от других видов бустинга заключается в поиске порогов для разбиения деревьев на узлы при помощи производных.

Как указано в документации, LightGBM является улучшением алгоритма градиентного бустинга с точки зрения *эффективности*, *скорости*, поддержки распределенной *параллельной обработки* и *GPU*.
LightGBM подходит для использования, если вы хотите построить модель с большим количеством данных. Если у вас всего 100 точек, лучше использовать другой алгоритм машинного обучения, так как в этом случае возможно переобучение.
В [документации  LightGBM](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) обращают внимание на следующие важные параметры:

- max_depth: максимальная глубина дерева

- num_leaves: количество листьев в дереве; рекомендуется настривать его меньше, чем 2^(max_depth) (например, 2^(max_depth)/1.5)

- min_data_in_leaf: минимальное количество наблюдений, которое может иметь лист дерева - маленькое значение приводит к
слишком глубокому и переобученному дереву. Рекомендуется устанавливать на уровне 3,4 - значных числе

- learning_rate: размер шага для каждой итерации при движении к минимуму функции потерь в градиентном спуске.

- feature_fraction: доли признаков/параметров, которые будут случайным образом выбираться на каждой итерации для построения деревьев

- bagging_fraction: задает долю от выборки, которая будут использоваться в каждой итерации для создания нового набора данных

- min_split_gain: минимальное значение прироста информации при разделения дерева. Увеличение данного параметра может
ускорить обучение, так как не любой прирост информации может быть важен для нас.

- num_iterations: количество базовых алгоритмов. Измененять лучше одновременно с *learning_rate*

- reg_lambda: коэффициент L1 регуляризации для решения проблемы переобучения и выбора признаков

- reg_alpha: коэффициент L2 регуляризации для решения проблемы переобучения и выбора признаков

## Level-wise (depth-first) tree growth

В данном подходе дерево строится рекурсивно до тех пор, пока не будет достигнута ***максимальная глубина***.

![](https://i.stack.imgur.com/e1FWe.png)



## XGboost

Самым популярным алгоритмом бустинга, как уже говорилось, является [XGboost](https://xgboost.readthedocs.io/en/latest/).

![](https://miro.medium.com/max/700/1*l4PN8hyAO4fMLxUbIxcETA.png)


[Xgboost](https://github.com/dmlc/xgboost) использует еще больше параметров для регуляризации базовых деревьев.

Целевая функция для оптимизации в Xgboost состоит из двух слагаемых: специфичной пункции потерь и регуляризатора $\Omega (f_k)$ для каждого из $K$ деревьев, где $f_k$ - прогноз $k$-ого дерева.


$$
obj(\theta) = \sum_{i}^{\ell} l(y_i - \hat{y_i}) +  \sum_{k=1}^{K} \Omega (f_k)
$$

Функция потерь зависит от решаемой задачи (Xgboost адаптирован под задачи классификации, регрессии и ранжирования, подробней хорошо описано в [документации](http://xgboost.readthedocs.io/) Xgboost), а регуляризатор выглядит следующим образом:

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T}w_j^2
$$

Первое слагаемое ($\gamma T$) штрафует модель за большое число листьев $T$, а второе ($\frac{1}{2} \lambda \sum_{j=1}^{T}w_j^2$) контролирует сумму весов модели в листьях. 

Добавим также, что сама процедура подбора деревьев устроена так, что в функции, при помощи которой подбираются признаки для разбиения, уже встроены первая и вторая производная исходной функции потерь. 

## Основные различия между CatBoost, LightGBM, XGBoost

### Расщепление узлов 

![](https://miro.medium.com/max/700/1*E006sjlIjabDJ3jNixRSnA.png)

Перед обучением все алгоритмы создают пары признак-сплит для всех факторов. Например, (возраст, <5), (возраст, >10), (сумма, >500). Эти пары признак-сплит строятся на основе гистограммы и используются во время обучения в качестве возможных расщеплений узлов. Этот метод предварительной обработки быстрее, чем точный жадный алгоритм, который линейно перечисляет все возможные разбиения для непрерывных признаков (как в классических деревьях решений).

**LightGBM** предлагает градиентное одностороннее сэмплирование (*GOSS*), которое выбирает разбиения, используя все наблюдения с большими значениями градиентов (т.е. с большой ошибкой) и случайную выборку наблюдений с малыми значениями градиентов. Чтобы сохранить одинаковое распределение данных при вычислении информационного прироста, *GOSS* вводит постоянный множитель для наблюдений данных с малыми градиентами. Таким образом, GOSS достигает хорошего баланса между увеличением скорости за счет уменьшения количества наблюдений и сохранением точности для обученных деревьев решений. Этот метод не является методом по умолчанию для LightGBM, поэтому его следует выбирать явно.

**Catboost** предлагает новую технику под названием Minimal Variance Sampling (*MVS*), которая представляет собой взвешенное сэмплирование Stochastic Gradient Boosting. В этой технике взвешенное сэмплирование происходит на уровне дерева, а не на уровне разбиения. Наблюдения для каждого дерева отбираются таким образом, чтобы максимизировать точность оценки разбиения.

**XGboost** не использует никаких методов взвешенной выборки, что делает его процесс разбиения более медленным по сравнению с *GOSS* и MVS.

### Как растут листья из узлов

**Catboost** строит сбалансированное дерево. На каждом уровне такого дерева выбирается пара признак-сплит, которая приносит наименьшие потери (согласно функции потерь) и используется для всех узлов данного уровня. Можно изменить с помощью параметра *grow-policy*.

**LightGBM** использует рост дерева по листьям (по принципу "лучший-первый"). Выбирается тот лист, который минимизирует функцию потерь, допуская рост несбалансированного дерева. Поскольку дерево растет не по уровням, а по листьям, при небольшой выборке может произойти переобучение. В таких случаях важно контролировать глубину дерева.

**XGboost** расщепляет узлы до заданного гиперпараметра *max_depth*, а затем начинает обрезать дерево в обратном направлении и удаляет части, за пределами которых нет положительного прироста информации. Это работает, поскольку иногда за расщеплением без снижения функции потерь может следовать расщепление с уменьшением функции потерь. XGBoost также может выполнять рост дерева по листьям (как LightGBM).

### Важность признаков

**Catboost** имеет два способа вычисления важности признаков. 

Первый - *PredictionValuesChange*. Для каждого признака *PredictionValuesChange* показывает, насколько в среднем изменяется предсказание при изменении значения признака. Признак будет иметь большую важность, если изменение значения признака вызывает большое изменение в прогнозируемом значении. Это метод расчета важности признака по умолчанию для неранжированных метрик. 

Второй метод - *LossFunctionChange*. Этот тип важности признака может использоваться для любой модели, но особенно полезен для моделей ранжирования. Для каждого признака значение представляет собой разницу между значением функции потерь модели с этим признаком и без него. Поскольку переоценка модели без одного из признаков требует больших вычислительных затрат, эта модель строится приблизительно, используя исходную модель с этим признаком, удаленным из всех деревьев в ансамбле. 

**LightGBM** и **XGBoost** имеют два похожих метода. 

Первый - *Gain*, который представляет собой улучшение точности (или общий выигрыш), приносимое признаком узлам, на которых он находится. 

Второй метод имеет разное название в каждом пакете: *split* (LightGBM) и *Frequency/Weight* (XGBoost). Этот метод вычисляет относительное количество раз, когда определенный признак встречается во всех расщеплениях деревьев модели. Этот метод может быть необъективным для категориальных признаков с большим количеством категорий.

У **XGBoost** есть еще один метод, *reach*, который представляет собой относительное количество наблюдений, связанных с признаком. Для каждого признака мы подсчитываем количество наблюдений, используемых для выбора узла листа.

# Ансамбли: основные выводы

1. Случайный лес - деревья строятся параллельно на бутстрапированных подвыборках и полностью с ограничением на максимальную глубину. Результат агрегируется. Деревья некоррелируемые друг с другом. Число признаков для разбиения выбирается, как правило, равное квадратному корню из размерности признакового пространства. Проблема - вычислительно затратно.

2. Бустинг - деревья строятся последовательно, на каждой итерации снижая ошибку, которая получается из композиции предыдущих базовых алгоритмов. 

3. CatBoost хорошо работает с категориальными признаками. На каждом разбиении использует один признак. Относительно быстрый.

4. LightGBM не ограничивает глубину, но ограничивает количество листьев. Самый быстрый алгоритм бустинга на всем диком мире мэшин лернинга.

5. XGBoost - чемпион по увеличению скора на 0.0000001 сотых.

 - **Catboost** - однаковые признаки для каждого уровня разбиения

 - **LightGBM** - ставим ограничение на количество листьев 

 - **XGboost** - ограничение на максимальную глубину дерева и обрезка в обратном направлении
